###Ingest constructors.json file

In [0]:
dbutils.widgets.text('p_data_source','')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#####Step 1 - Read the JSON file using the spark dataframe reader

In [0]:
constructors_schema = "constructorId INT,constructorRef STRING,name STRING,nationality STRING,url STRING"

In [0]:
constructors_df = spark.read.json(f"{raw_folder_path}/constructors.json",schema=constructors_schema)

#####Step 2 - Drop unwanted columns from the dataframe

In [0]:
from pyspark.sql.functions import col

In [0]:
constructors_dropped_df = constructors_df.drop(col('url'))

#####Step 3 - Rename columns and add ingestion date

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
constructors_final_df = constructors_dropped_df.withColumnRenamed('constructorId','constructor_id') \
                                                .withColumnRenamed('constructorRef','constructor_ref') \
                                                .withColumn('ingestion_date',current_timestamp()) \
                                                .withColumn('data_source',lit(v_data_source))

#####Step 4 - Write the output to parquet file

In [0]:
constructors_final_df.write.mode('overwrite').parquet(f"{processed_folder_path}/constructors")

In [0]:
dbutils.notebook.exit('Success')